## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [49]:
path = '../03-corpus/2-data/1-fr/'
acteur = "msss"

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=',')[["url", "text"]]

csv

,url,text
0,https://www.msss.gouv.qc.ca/repertoires/indica...,Durée moyenne de séjour pour la clientèle de 7...
1,https://www.msss.gouv.qc.ca/repertoires/indica...,Nombre de personnes ayant reçu des services en...
2,https://www.msss.gouv.qc.ca/inc/documents/mini...,le groupe de médecine de famille (gmf) ...
3,https://www.msss.gouv.qc.ca/ministere/salle-de...,Pandémie de la COVID-19 – Le gouvernement du Q...
4,https://www.msss.gouv.qc.ca/ministere/salle-de...,Pandémie de la COVID-19 – Mise en ligne de nou...
...,...,...
9742,https://www.msss.gouv.qc.ca/repertoires/indica...,Pourcentage d'examens électifs primaires en im...
9743,https://www.msss.gouv.qc.ca/repertoires/indica...,Répertoire des indicateurs de gestion en santé...
9744,https://www.msss.gouv.qc.ca/professionnels/soi...,Aspects légaux - Santé mentale - Professionnel...
9745,https://www.msss.gouv.qc.ca/professionnels/vac...,Anaphylaxie - Urgences liées à la vaccination ...


In [50]:
liste = [lien.replace('//', "/") for lien in csv['url'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien and not 'nouvelles' in lien and not 'pdf' in lien and not 'gaspesie' in lien]
liste 

['https:/www.msss.gouv.qc.ca/repertoires/indicateurs-gestion/indicateur-000009/?&date=DESC&titre=ASC&statut=officiel&critere=statut\n',
 'https:/www.msss.gouv.qc.ca/repertoires/indicateurs-gestion/indicateur-000202/?&date=DESC\n',
 'https:/www.msss.gouv.qc.ca/ministere/salle-de-presse/communique-3043/\n',
 'https:/www.msss.gouv.qc.ca/ministere/salle-de-presse/communique-2857/\n',
 'https:/www.msss.gouv.qc.ca/repertoires/indicateurs-gestion/recherche?page=2&date=ASC\n',
 'https:/www.msss.gouv.qc.ca/professionnels/promotion-de-la-sante/saines-habitudes/documents-et-publications/\n',
 'https:/www.msss.gouv.qc.ca/repertoires/indicateurs-gestion/indicateur-000023/?&date=DESC&titre=DESC&prog=deficience-intellectuelle-et-ted&critere=prog\n',
 'https:/www.msss.gouv.qc.ca/ministere/salle-de-presse/sujet-255/\n',
 'https:/www.msss.gouv.qc.ca/professionnels/statistiques-donnees-sante-bien-etre/statistiques-de-sante-et-de-bien-etre-selon-le-sexe-volet-national/taux-de-mortalite-par-suicide/\n',
 '

In [51]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens = []
for i in range (2, 9): # On prend les liens de niveau 2 à 8
    try:
        liens += [lien for lien in liste if len(lien.split('/')) > i]
    except Exception as e:
        True

In [52]:
regroupements = {}

for lien in liens:
    for i in range(3,9):
        try:
            tag = lien.split('/')[i-1]
            if tag in regroupements:
                regroupements[tag].append(lien)
                
            else:
                if len(tag)>1:
                    regroupements[tag] = []

        except Exception as e:
            True

tags = list(regroupements.keys())

In [53]:
tags

['repertoires',
 'indicateurs-gestion',
 'indicateur-000009',
 '?&date=DESC&titre=ASC&statut=officiel&critere=statut\n',
 'indicateur-000202',
 '?&date=DESC\n',
 'ministere',
 'salle-de-presse',
 'communique-3043',
 'communique-2857',
 'recherche?page=2&date=ASC\n',
 'professionnels',
 'promotion-de-la-sante',
 'saines-habitudes',
 'documents-et-publications',
 'indicateur-000023',
 '?&date=DESC&titre=DESC&prog=deficience-intellectuelle-et-ted&critere=prog\n',
 'sujet-255',
 'statistiques-donnees-sante-bien-etre',
 'statistiques-de-sante-et-de-bien-etre-selon-le-sexe-volet-national',
 'taux-de-mortalite-par-suicide',
 'sante-environnementale',
 'pollens',
 'travailler-en-concertation',
 'communique-2829',
 'communique-3032',
 'indicateur-000033',
 '?\n',
 'soins-et-services',
 'guide-urgences-ressources-information-technologies-et-installations',
 'dossier-sante-quebec-dsq',
 'avis-de-grossesse-provincial-informatise-ma-grossesse',
 'indicateur-000058',
 '?&date=ASC&prog=deficience-int

In [38]:
# On garde seulement les sections qui contiennent au moins 5 sous pages
tags_retenus = [tag for tag in tags if len(regroupements[tag]) >=30]

In [39]:
print('Ça nous donne {} tags.'.format(len(tags_retenus)))
tags_retenus

Ça nous donne 310 tags.


['repertoires',
 'indicateurs-gestion',
 'indicateur-000009',
 '?&date=DESC&titre=ASC&statut=officiel&critere=statut\n',
 'indicateur-000202',
 '?&date=DESC\n',
 'ministere',
 'salle-de-presse',
 'professionnels',
 'promotion-de-la-sante',
 'indicateur-000023',
 'statistiques-donnees-sante-bien-etre',
 'statistiques-de-sante-et-de-bien-etre-selon-le-sexe-volet-national',
 'sante-environnementale',
 'pollens',
 'indicateur-000033',
 'soins-et-services',
 'indicateur-000058',
 'indicateur-000197',
 '?&date=DESC&prog=deficience-intellectuelle-et-tsa&critere=prog\n',
 'vision-de-la-saine-alimentation',
 'indicateur-000077',
 'guide-urgences-ressources-humaines',
 'documents',
 'surveillance-de-la-sante-maternelle-et-foeto-infantile',
 'vaccination',
 'piq-calendriers-de-vaccination',
 'indicateur-000165',
 '?&date=DESC&statut=officiel&critere=statut\n',
 'dependances',
 'indicateur-000132',
 '?&date=DESC&titre=DESC\n',
 'statistiques-de-sante-et-de-bien-etre-selon-le-sexe-volet-regional',


In [40]:
import os, shutil
from pathlib import Path
from slugify import slugify

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../03-corpus/2-sous-corpus/'

#Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags_retenus:
    file_name =  acteur + "_" + slugify(str(tag).strip('\n'))

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    try:
        data = csv[csv["url"].str.match('.*[\-\_\/]'+tag+'.*')]
        data.to_csv(folder_path + file_name + ".csv") #encoding='ISO-8859-1'
    except Exception as e:
        print(tag, str(e))
                    

In [41]:
with open(folder_path + acteur + '_sous-corpus.txt', 'w') as f:
    for file in os.listdir(folder_path):
        f.write(file + '\n')

Après avoir manuellement indexé

In [42]:
from pandas import read_csv
filepath = '../03-corpus/2-sous-corpus/0-semi-automatic_tags_' + acteur + '.csv'
with open(filepath, 'r', encoding = 'utf-8') as f:
    csv = read_csv(f, sep=';')
    
csv = csv[csv['oui/non'] != 'x']
csv = csv[csv['oui/non'] != 'X']

csv

,sous-corpus,oui/non
3,msss_activites-et-services-sante-dentaire.csv,Dentistry
4,msss_aines.csv,Aînés
5,msss_alcool-drogues-jeu.csv,Dependances
6,msss_alzheimer-et-autres-troubles-neurocogniti...,Neurologie
7,msss_amis-des-bebes.csv,Obstetrics
...,...,...
304,msss_vaccination.csv,Vaccination
305,msss_vieillir-et-vivre-ensemble.csv,Programmes
306,msss_virus-du-nil-occidental-vno.csv,Maladies / infections
307,msss_vision-de-la-saine-alimentation.csv,Habitudes de vie


In [43]:
tags = list(set([tag for tag in csv['oui/non'].tolist()]))

tags

['Biologie médicale',
 'Zoonoses',
 'Maladies / infections',
 'Populations',
 'Itinérance',
 'Sante mentale',
 'Soins et services',
 'Habitudes de vie',
 'Soins paliatifs',
 'Dependances',
 'Réseau santé',
 'Cancer',
 'Environnement',
 'covid-19',
 'Jeunesse',
 'Urgence',
 'Services communautaires',
 'Dentistry',
 'Pédiatrie',
 'Chaleur',
 'Promotion de la santé',
 'Neurologie',
 'Programmes',
 'DI / TSA',
 'Laboratoires',
 'Ethique',
 'Obstetrics',
 'Immunologie',
 'Hygiène',
 'Traumatologie',
 'DeficienceHandicap',
 'Maladies chroniques',
 'Services sociaux',
 'Aînés',
 'Santé Publique',
 'Vaccination']

In [46]:
regroupements = {tag : [] for tag in tags}
for tag in tags:
        tuples = csv[csv['oui/non'] == tag]
        liste = [t.strip('\n') for t in tuples["sous-corpus"].tolist()]
        regroupements[tag] = liste


In [47]:
regroupements

{'Biologie médicale': ['msss_biologie-medicale.csv'],
 'Zoonoses': ['msss_zoonoses.csv'],
 'Maladies / infections': ['msss_flash-grippe.csv',
  'msss_grippe.csv',
  'msss_mado.csv',
  'msss_maladie-lyme.csv',
  'msss_maladies-a-declaration-obligatoire.csv',
  'msss_maladies-infectieuses.csv',
  'msss_prevention-infections-services-de-garde-ecoles.csv',
  'msss_virus-du-nil-occidental-vno.csv'],
 'Populations': ['msss_informations-geographiques-et-de-population.csv',
  'msss_particularites-des-services-aux-communautes-autochtones.csv'],
 'Itinérance': ['msss_colloque-national-en-itinerance.csv',
  'msss_itinerance.csv'],
 'Sante mentale': ['msss_date-asc-prog-sante-mentale-critere-prog.csv',
  'msss_date-desc-prog-sante-mentale-critere-prog.csv',
  'msss_date-desc-titre-asc-prog-sante-mentale-critere-prog.csv',
  'msss_date-desc-titre-desc-prog-sante-mentale-critere-prog.csv',
  'msss_guide-urgences-sante-mentale.csv',
  'msss_journees-annuelles-de-sante-mentale.csv',
  'msss_sante-ment

In [48]:
import pandas
from slugify import slugify

# Créer de nouveaux CSV contenant les données regroupées selon le tag qui leur est associé
path='../03-corpus/2-sous-corpus/'

for tag in regroupements:

    # Aller chercher les éléments correspondant au sous-corpus dans le csv principal et les copier dans un nouveau CSV 
    dfs = []
    path='../03-corpus/2-sous-corpus/'
    for doc in regroupements[tag]:
        with open(path + doc, 'r', encoding='utf-8') as f:
            csv = read_csv(f)
            dfs.append(csv)

    # Concatenate all data into one DataFrame
    try:
        path='../03-corpus/2-data/1-fr/' + acteur +'/'
        file_path = path + acteur + '_'  + slugify(str(tag)) + '.csv'
        big_frame = pandas.concat(dfs, ignore_index=True)
        big_frame.to_csv(file_path)
        print(tag + "\t" + str(len(big_frame)))

    except Exception as e:
        print(tag, str(e))



Biologie médicale	9
Zoonoses	42
Maladies / infections	152
Populations	21
Itinérance	79
Sante mentale	207
Soins et services	269
Habitudes de vie	129
Soins paliatifs	17
Dependances	177
Réseau santé	5120
Cancer	212
Environnement	159
covid-19	82
Jeunesse	76
Urgence	21
Services communautaires	12
Dentistry	53
Pédiatrie	7
Chaleur	261
Promotion de la santé	52
Neurologie	18
Programmes	142
DI / TSA	165
Laboratoires	19
Ethique	194
Obstetrics	146
Immunologie	54
Hygiène	9
Traumatologie	8
DeficienceHandicap	41
Maladies chroniques	46
Services sociaux	22
Aînés	199
Santé Publique	4688
Vaccination	504
